<a href="https://colab.research.google.com/github/ashwin2k/Titanic-Dataset/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Titanic Dataset Prediction using Machine Learning**

---



In [ ]:
from google.colab import files

In [ ]:
train_dataset=files.upload()

Saving train (1).csv to train (1).csv


In [ ]:
test_dataset=files.upload()

Saving test (1).csv to test (1).csv


In [ ]:
import pandas as pd
import numpy as np
import io
 
#For feature selection--
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
train_data=pd.read_csv(io.BytesIO(train_dataset['train (1).csv']))

In [ ]:
test_data=pd.read_csv(io.BytesIO(test_dataset['test (1).csv']))

After converting into Dataframes, we use the head() method to see how the data is organized.

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Also by using the built-in describe() method, we can get a general descriptive statistics of our data

In [ ]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


The shape method gives the dimentions of our data. As we can see from the output, the dataframe as 891 rows and 12 features.

In [ ]:
train_data.shape

(891, 12)

#**Replacing null/NaN values**

---
The first important step before training our model is to remove or replace the null values. For this purpose, I create a simple helper function that counts the number of null values in each column/feature.


In [ ]:
#helper_functions
def countnan(df):
  return df.isnull().sum().sort_values(ascending = False)

In [ ]:
countnan(train_data)

Cabin          687
Age            177
Embarked         2
Fare             0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
Survived         0
PassengerId      0
dtype: int64

As we see, there are 3 columns with null values. Age is a continuous variable, so i use the mean to replace null values. For others, i use the *forwardfill* (ffill) method.

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_data['Age'].mean()

29.69911764705882

In [ ]:
train_data['Embarked'].fillna(method='ffill',inplace=True)
train_data['Cabin'].fillna(method='ffill',inplace=True)
train_data['Age'].fillna(train_data['Age'].mean(),inplace=True)

Since, the first value of *'Cabin'* variable itself is null, we cant use ffill there since there is no previous value. So, I manually fill the value for it.

In [ ]:
train_data.iloc[0,10]='C85'

In [ ]:
countnan(train_data)

Embarked       0
Cabin          0
Fare           0
Ticket         0
Parch          0
SibSp          0
Age            0
Sex            0
Name           0
Pclass         0
Survived       0
PassengerId    0
dtype: int64

Now, we can see that there are no null values anymore.


---

#The next step of preprocessing is dropping unwanted variables, like *Name*

In [ ]:
final_data=train_data.drop(['Name','Ticket'],axis=1)

In [ ]:
final_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,C85,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,C85,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,C123,S


We now proceed with feature selection, where we optimally select the best variables. I used the *SelectKBest* to select the best variables that contribute most to the ouput variable. I use the chi-squared statistical test to select the best features.

In [ ]:
X=final_data.iloc[:,2:]

In [ ]:
Y=final_data.iloc[:,1]

In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k=10)

But before that, I need to convert the categorical variables to integers so that the computer can understand. I **One-Hot Encoding** for this purpose.

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit_transform(final_data['Sex'])

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,

In [ ]:
test_copy=final_data

In [ ]:
test_copy['Sex']=label_encoder.fit_transform(test_copy['Sex'])

In [ ]:
test_copy['Cabin']=label_encoder.fit_transform(test_copy['Cabin'])
test_copy['Embarked']=label_encoder.fit_transform(test_copy['Embarked'])

In [ ]:
test_copy.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,7.2500,81,2
1,2,1,1,0,38.0,1,0,71.2833,81,0
2,3,1,3,0,26.0,0,0,7.9250,81,2
3,4,1,1,0,35.0,1,0,53.1000,55,2
4,5,0,3,1,35.0,0,0,8.0500,55,2


In [ ]:
X=test_copy.iloc[:,2:]
Y=test_copy.iloc[:,1]

In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k='all')

In [ ]:
fit = bestfeatures.fit(X,Y)

We know fit the SelectKBest class and extract the largest 10 features along with their scores.

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))

      Specs        Score
5      Fare  4518.319091
1       Sex    92.702447
0    Pclass    30.873699
2       Age    24.687926
7  Embarked    10.893210
4     Parch    10.097499
3     SibSp     2.581865
6     Cabin     0.013113


We see that the Cabin feature has no significance at all and having it only leads to overfitting of data. So we can drop it.

In [ ]:
train_data=pd.read_csv(io.BytesIO(train_dataset['train (1).csv']))

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_data['Embarked'].fillna(method='ffill',inplace=True)
train_data['Cabin'].fillna(method='ffill',inplace=True)
train_data['Age'].fillna(train_data['Age'].mean(),inplace=True)

In [ ]:
data=train_data.drop(['Name','Ticket'],axis=1)

In [ ]:
data.iloc[0,8]='C85'

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,C85,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,C85,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,C123,S


In [ ]:
data=data.drop(['Cabin'],axis=1)

In [ ]:
X_features=['Sex','Embarked','Pclass','Age','SibSp','Parch','Fare']

In [ ]:
X=pd.get_dummies(data[X_features])

In [ ]:
Y=data['Survived']

In [ ]:
X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,35.000000,1,0,53.1000,1,0,0,0,1
4,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,0,1,0,0,1
887,1,19.000000,0,0,30.0000,1,0,0,0,1
888,3,29.699118,1,2,23.4500,1,0,0,0,1
889,1,26.000000,0,0,30.0000,0,1,1,0,0


In [ ]:
Y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
test_data['Embarked'].fillna(method='ffill',inplace=True)
test_data['Cabin'].fillna(method='bfill',inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)

In [ ]:
countnan(test_data)

Cabin          3
Fare           1
Embarked       0
Ticket         0
Parch          0
SibSp          0
Age            0
Sex            0
Name           0
Pclass         0
PassengerId    0
dtype: int64

In [ ]:
test_data['Cabin'].fillna(method='ffill',inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

In [ ]:
test=test_data.drop(['Cabin','Name','Ticket'],axis=1)

In [ ]:
X_test=pd.get_dummies(test_data[X_features])

We can know start fitting our train data. I used the RandomForestClassifier for this purpose. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
 
logisticreg=LogisticRegression()
model = RandomForestClassifier(n_estimators=550,criterion='gini',max_depth=10)
model.fit(X, Y)
Y_pred = model.predict(X_test)
model.score(X,Y)

0.9427609427609428

To know more about how the parameters are choosen,validation accuracy, etc. see the other notebook.